# Process FASTA files
- create theoretically considered peptides considere by search engines
- dump results

> Based on notebook received by [Annelaura Bach](https://www.cpr.ku.dk/staff/mann-group/?pure=en/persons/443836) and created by Johannes B. Müller \[[scholar](https://scholar.google.com/citations?user=Rn1OS8oAAAAJ&hl=de), [MPI Biochemistry](https://www.biochem.mpg.de/person/93696/2253)\]

In [ ]:
import json
import logging
import os

import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
from vaep.fasta import cleave_to_tryptic

## Core Functionality - Example

- write tests for core functinality
- refactor to file

In [ ]:
test_data = {
    "meta": ">tr|A0A024R1R8|A0A024R1R8_HUMAN HCG2014768, isoform CRA_a OS=Homo sapiens OX=9606 GN=hCG_2014768 PE=4 SV=1",
    "seq": "MSSHEGGKKKALKQPKKQAKEMDEEEKAFKQKQKEEQKKLEVLKAKVVGKGPLATGGIKKSGKK",
    "peptides": ["MSSHEGGK", "EMDEEEK", "GPLATGGIK"],
}

regex is slower than native string replacing and splitting in Python

In [ ]:
# import re
# cut_by_trypsin = re.compile('([^K]+K)|([^R]+R)')
# _res = cut_by_trypsin.split(test_data['seq'])
# [_pep for _pep in _res if _pep != '' and _pep != None]


- map peptide set of peptides (how to deal with mis-cleavages?)
    - mis-cleavages can happen both to the peptide before and after.
    > `pep1, pep2, pep3, pep4, pep5`  
    > `pep1pep2, pep2pep3, pep3pep4, pep4pep5`
    - sliding windows can pass trough the list of peptides - should work with recursion

In [ ]:
l_peptides = test_data["seq"].replace("K", "K ").replace("R", "R ").split()
l_peptides

`add_rxk` should add pattern of starting R and trailing K ?  

In [ ]:
last_pep = ""
temp_peps = []
num_missed_cleavages = 1
add_rxk = True

sec_last_pep = ""

pep_rdx = []

for pep in l_peptides:
    if last_pep != "":
        temp_peps.append(last_pep + pep)
    if add_rxk and sec_last_pep != "" and len(sec_last_pep) <= 2:
        _pep_rxk = sec_last_pep + last_pep + pep
        print(_pep_rxk)
        pep_rdx.append(_pep_rxk)
        temp_peps.append(_pep_rxk)

    sec_last_pep = last_pep  # sec_last_pep, last_pep = last_pep, pep ?
    last_pep = pep
temp_peps

In [ ]:
repr(pep_rdx)

Missed cleavages core functionality (adapted)

In [ ]:
example_peptides_fasta = cleave_to_tryptic(
    test_data["seq"], num_missed_cleavages=2, add_rxk=True
)
print("number of peptides: ", [len(_l) for _l in example_peptides_fasta])
example_peptides_fasta[-1]

In [ ]:
print("".join(example_peptides_fasta[0]), *example_peptides_fasta, sep="\n")

rdx peptides are a subset of two missed cleavage sites peptides. There are omitted when two and more cleavage site can be skipped.

In [ ]:
example_peptides_fasta = cleave_to_tryptic(
    test_data["seq"], num_missed_cleavages=1, add_rxk=True
)
print("number of peptides: ", [len(_l) for _l in example_peptides_fasta])
example_peptides_fasta[-1]

Data Structure is no a list of list. Maybe this could be improved. Information what kind of type the peptide is from, is still interesting.

### Original Code

now adapted in `vaep.fasta.py`

In [ ]:
# def cleave_to_tryptic(seq="", num_missed_cleavages=1, reversed=False):
# #    Takes a sequence and returns an array of peptides cleaved C-term to R  and K
# #    number of missed cleavages can be 0 or 1
# #    works with single letter sequences
# #    :param seq:
# #    num_missed_cleavges,
# #    reversed: boolean flag. If true, append the reversed peptide spequences as well
# #    :return: list of strings
#     if  0 < num_missed_cleavages > 2:
#         raise NotImplementedError("`num_missed_cleavages` of missed cleavages can be 0 or 1")
#     if seq == "":
#         return
#     add_rxk = True  # flag for adding bonus peptides at patterns like KXR, which trypsin doesn't cleave efficiently

#     seq.replace(' ', '')  # sequence must not contain whitespaces todo: all whitespaces
#     seq = seq.upper()
#     seq = seq.replace('K', 'K ').replace('R', 'R ')  # introduce a white space so we can split there
#     pep_seqs = seq.split()

#     last_pep = ""
#     temp_peps = []
#     if num_missed_cleavages == 0 and add_rxk:
#         for pep in pep_seqs:
#             if 0 < len(last_pep) <= 2:
#                 temp_peps.append(last_pep+pep)
#             last_pep = pep
#         pep_seqs += temp_peps

#     sec_last_pep = ""


#     if num_missed_cleavages == 1:  # add joined peptides
#         for pep in pep_seqs:
#             if last_pep != "":
#                 temp_peps.append(last_pep + pep)
#             if add_rxk and sec_last_pep != "" and len(sec_last_pep) <= 2:
#                 temp_peps.append(sec_last_pep + last_pep + pep)
#             sec_last_pep = last_pep  # sec_last_pep, last_pep = last_pep, pep ?
#             last_pep = pep
#         pep_seqs += temp_peps
#     # # if num_missed_cleavages == 2: todo: implement two missed cleavages
#     # # add recursion to build peptides iteratively while iterating through peptide sequences


#     if reversed:
#         reversed_seqs = []
#         for pep in pep_seqs:
#             if str(pep).endswith('K'):  # change this to change the MW of the peptide
#                 pep = pep[0:-1] + 'r'
#             elif str(pep).endswith('R'):
#                 pep = pep[0:-1] + 'k'
#             rev_pep = pep[::-1]
#             reversed_seqs.append(rev_pep)

#         pep_seqs += reversed_seqs

#     return pep_seqs

Header is used to distinguish new entries in fasta files:

In [ ]:
# # #ToDo: Check if file-reader
# def read_fasta(fp):
#     """Read a fasta file and yield continously header and sequences."""
#     header, seq = None, []
#     for line in fp:
#         line = line.rstrip()
#         if line.startswith(">"):
#             if header: yield (header, ''.join(seq))
#             header, seq = line, []
#         else:
#             seq.append(line)
#     if header: yield (header, ''.join(seq))

Flattens a list of list using generators:

In [ ]:
# def iterFlatten(root):
#     """Flatten a nested structure."""
#     if isinstance(root, (list, tuple)):
#         for element in root:
#             for e in iterFlatten(element):
#                 yield e
#     else:
#         yield root

## Process Fasta Files

First define input Folder and the file location of the created peptides:

In [ ]:
from src.file_utils import search_files

fasta_files = search_files(path=".", query=".fasta")
print("\n".join(fasta_files.files))

### Define Setup

Set input FASTA, Output .txt name, lower legth cutoff, missed cleavages and if to report reverse. 

Tryptic digest of Fastas to Peptides >6 in list for matching with measured peptides  

In [ ]:
CUTOFF_LEN_PEP = 7
MAX_MISSED_CLEAVAGES = 2  # default in MaxQuant
DECOY_REVERSE = False
SUMMARY_FILE = "tex/fasta_tryptic_analysis.tex"

_summary_text = (
    "The theoretical analysis of the fasta files gives an idea about how many possible peptides \n"
    "can be expected by cleaving proteins by peptides. The hyperparameters for peptide creation are \n"
    f"to consider the minimal peptide length to be {CUTOFF_LEN_PEP} amino acids, \n"
    f"to consider a maximum of {MAX_MISSED_CLEAVAGES} missed cleavage sites (default in MaxQuant) and \n"
    f"to {'not ' if not DECOY_REVERSE else ''}add decoy peptides by reversing peptide sequences. \n"
)
print(_summary_text, sep="\n")

From the [Fasta Meta information](https://ebi14.uniprot.org/help/fasta-headers) the Identifier is extracted.

```
>db|UniqueIdentifier|EntryName ProteinName OS=OrganismName OX=OrganismIdentifier [GN=GeneName ]PE=ProteinExistence SV=SequenceVersion
```
- db is `sp` for UniProtKB/Swiss-Prot and `tr` for UniProtKB/TrEMBL.
- `UniqueIdentifier` is the primary *accession number* of the UniProtKB entry. (seems to be used by MQ)
- `EntryName` is the entry name of the UniProtKB entry.
- `ProteinName` is the recommended name of the UniProtKB entry as annotated in the *RecName* field. For UniProtKB/TrEMBL entries without a *RecName* field, the *SubName* field is used. In case of multiple SubNames, the first one is used. The 'precursor' attribute is excluded, 'Fragment' is included with the name if applicable.

`>tr` or `>sp`

In [ ]:
from vaep.fasta import read_fasta

data_fasta = {}

for _fasta in tqdm(fasta_files.files):

    with open(_fasta) as fp:
        for i, (metainfo, seq) in tqdm(enumerate(read_fasta(fp))):
            identifier = metainfo.split("|")[1]
            if identifier in data_fasta:
                raise ValueError("Key seen before: {}".format(identifier))
            _all_peptides = cleave_to_tryptic(
                seq, num_missed_cleavages=MAX_MISSED_CLEAVAGES, reversed=DECOY_REVERSE
            )
            data_fasta[identifier] = {
                "meta": metainfo,
                "seq": seq,
                "peptides": [
                    [_pep for _pep in _peptides if len(_pep) >= CUTOFF_LEN_PEP]
                    for _peptides in _all_peptides
                ],
            }

`fasta_data` holds all information to pick a subset of peptides from peptides intensity tables

In [ ]:
# from random import sample
# sample_ids = sample(list(data_fasta), 10)
# for _id in sample_ids:
#     print("Unique Identifier: {}: \n\t AA-Seq: {} \n\t Header: {} \n\t Peptides: {}\n".format(_id, data_fasta[_id]['seq'], data_fasta[_id]['meta'], data_fasta[_id]['peptides']))
data_fasta["A0A024R1R8"]

### Proteins' Isoforms

Possible to join "isoforms" by joining all variants to one. Isoforms are numbered from the second on by appending `-i` for $i>1$, i.e. starting with `-2`. The gene name of which the protein (isoform) originate can be obtained by using [id mapping](https://www.uniprot.org/help/api_idmapping). Isoforms are not mapped automatically by Uniprot to its GENENAME, i.e. you have to strip all `-i`, e.g `-2`, `-3`, for querying. Here the protein, gene pairs are mapped to the unique protein identifiers.

In [ ]:
prot_ids = list(data_fasta.keys())
prot_ids = pd.Series(prot_ids)
prot_ids

In [ ]:
mask = prot_ids.str.contains("-")
isoforms = prot_ids.copy().loc[mask]
isoforms

In [ ]:
N_prot_with_isoform = isoforms.str.split("-").str[0].nunique()

In [ ]:
n_unique_proteins_wo_isoforms = len(prot_ids) - len(isoforms)
_summary_text += "\nA total of {} proteins have at least one more isoform. ".format(
    N_prot_with_isoform
)
_summary_text += f"Collapsing isoforms into one protein results in {n_unique_proteins_wo_isoforms} proteins."
print(_summary_text)

Remove Isoforms from list. How to integrate this information before?

fasta-data has to be merge one-to-many.

In [ ]:
id_map = pd.DataFrame(
    prot_ids.str.split("-").str[0], columns=["protein"]
)  # , index=list(prot_ids))
id_map.index = pd.Index(prot_ids, name="prot_id")
id_map

In [ ]:
id_map.loc[id_map.index.str.contains("A0A096LP49|Q9Y6Z5|W5XKT8")]

In [ ]:
l_proteins = id_map.protein.unique()
print(
    f"There are {len(l_proteins)} unique proteins without isoforms listed in the used fasta files."
)
# Check with pervious result.
assert n_unique_proteins_wo_isoforms == len(l_proteins)

In [ ]:
from config import FN_PROT_GENE_MAP

from vaep.uniprot import query_uniprot_id_mapping

try:
    with open(FN_PROT_GENE_MAP) as f:
        dict_protein_to_gene = json.load(f)
    logging.warning(f"Loaded pre-cached map dict_protein_to_gene: {FN_PROT_GENE_MAP}")
except FileNotFoundError:
    dict_protein_to_gene = {}
    start = 0
    for end in list(range(10000, len(l_proteins), 10000)):
        print(f"Retrieve items {start+1:6} to {end:6}")
        _id_to_gene = query_uniprot_id_mapping(l_proteins[start:end])
        print(f"Found {len(_id_to_gene)} gene names")
        dict_protein_to_gene.update(_id_to_gene)
        start = end
    print(f"Retrieve items {start:6} to {len(l_proteins):6}")
    _id_to_gene = query_uniprot_id_mapping(l_proteins[start:])
    print(f"Found {len(_id_to_gene)} gene names")
    dict_protein_to_gene.update(_id_to_gene)
    with open(FN_PROT_GENE_MAP, "w") as f:
        json.dump(dict_protein_to_gene, f, indent=4, sort_keys=False)

In [ ]:
genes = pd.Series(dict_protein_to_gene, name="gene")
genes

In [ ]:
assert (
    len(genes) == 72473
), f"The number of proteins associated to a gene found on 22.09.2020 was 72473, not it's {len(genes)}"

In [ ]:
id_map = id_map.merge(genes, how="outer", left_on="protein", right_index=True)
id_map.sort_values(by=["gene", "protein"], inplace=True)
id_map

In [ ]:
mask_no_gene = id_map.gene.isna()
id_map.loc[mask_no_gene]

In [ ]:
_summary_text += (
    f"\nThere are {id_map.gene.isna().sum()} protein IDs (or {id_map.loc[mask_no_gene].protein.nunique()} proteins) "
    "without a gene associated to them in the current faste files."
)
print(_summary_text)

### Isotopes mapping

Isotopes are mapped now to a protein with the same name. The same can be achieved by just discarding everything behind the hypen `-`

In [ ]:
id_map.loc[id_map.index.str.contains("-")]

Save id_map

In [ ]:
from config import FN_ID_MAP

id_map.to_json(FN_ID_MAP, orient="split", indent=4)

### Most proteins with a missing gene are deleted

If one checks manually some of the examples (e.g. the hundred provided here), one sees that all are deleted from Uniprot.

> How to obtain different versions of UniProt?!

In [ ]:
from vaep.utils import sample_iterable

if not len(dict_protein_to_gene) == len(l_proteins):
    print("Not all ids are mapped.")
    _diff = set(l_proteins).difference(dict_protein_to_gene.keys())
    print(f"Number of protein identifiers not mapped to a gene: {len(_diff)}")
    print(f'Look at {100} examples: {", ".join(sample_iterable(_diff, 100))}')

In [ ]:
_summary_text += (
    f"\nMost of the {len(_diff)} proteins ({len(_diff)/len(l_proteins)*100:.2f} percent of the unique proteins) "
    "not mapped to a gene name are deleted in the most current version of UniProt. "
    "The versioning of the fasta-files has to be investigated."
)
_summary_text += (
    f"\nProteins are mapped to a total number of genes of {id_map.gene.nunique()}.\n"
)
print(_summary_text)

In [ ]:
f"Proteins are mapped to a total number of genes of {len(set(dict_protein_to_gene.values()))}"

### Map peptide to either identifier, common protein or gene  


In [ ]:
from collections import defaultdict

from vaep.fasta import iterFlatten

peptide_to_prot = defaultdict(list)
for _id, _data in tqdm(data_fasta.items()):
    for _pep in iterFlatten(_data["peptides"]):
        peptide_to_prot[_pep].append(_id)

_summary_text += f"\nConsidering {MAX_MISSED_CLEAVAGES} missed cleavage site(s) there are {len(peptide_to_prot):,d} unique peptides."

In [ ]:
print(_summary_text)

In [ ]:
{_key: peptide_to_prot[_key] for _key in sample_iterable(peptide_to_prot.keys())}

In [ ]:
%time
from config import FN_PEP_TO_PROT

with open(FN_PEP_TO_PROT, "w") as f:
    json.dump(peptide_to_prot, f, indent=4, sort_keys=False)

### Plot histograms for different levels of abstraction

Plot counts of matched 
   1. protein IDs
   2. proteins (joining isoforms)
   3. genes
   
to their peptides. See how many unique peptides exist. The number of peptides should stay the same, so the counts do not have to be normalized.

In [ ]:
from vaep.fasta import count_peptide_matches

counters = {}
levels = ["protein_id", "protein", "gene"]
for level in levels:
    counters[level] = pd.Series(
        count_peptide_matches(peptide_to_prot, dict_protein_to_gene, level=level)
    )

In [ ]:
for level in levels:
    print(f"{level}: {counters[level]['AACLCFR']}")

In [ ]:
peptide_to_prot["AACLCFR"]

In [ ]:
_prots = {x.split("-")[0] for x in peptide_to_prot["AACLCFR"]}
{dict_protein_to_gene[_prot] for _prot in _prots}

In [ ]:
import numpy as np

from vaep.pandas import combine_value_counts

counts_by_level = combine_value_counts(pd.DataFrame(counters))
counts_by_level = counts_by_level.replace(np.nan, 0).astype(int)
counts_by_level

Interpretation: Peptides are assigned \# of times to a protein_id, protein or gene respectively.

Check that for all levels the same number of peptides are counted.

In [ ]:
counts_by_level.sum()

Plot the frequency of matched proteins to one peptide sequence:

In [ ]:
import matplotlib

fig, ax = plt.subplots(figsize=(13, 7))

ax = counts_by_level.iloc[:5].plot(kind="bar", ax=ax)
ax.set_ylabel("peptide counts")
ax.set_xlabel("number of matched levels")
# ax.yaxis.set_major_locator(matplotlib.ticker.MaxNLocator(integer=True))
# ax.yaxis.set_major_locator(matplotlib.ticker.FixedLocator(['0', '500,000', '1,000,000', '1,500,000', '2,000,000', '2,500,000', '3,000,000', '3,500,000']))
ax.yaxis.set_major_formatter("{x:,}")
# _y_ticks = ax.set_yticklabels(['0', '500,000', '1,000,000', '1,500,000', '2,000,000', '2,500,000', '3,000,000', '3,500,000'])
from vaep.io_images import _savefig

_savefig(fig, folder="figures", name="fasta_top4")

In [ ]:
ax.get_yticklabels()

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(17, 10))
axes = axes.flatten()

counts_by_level.iloc[:10].plot(kind="bar", ax=axes[0])
axes[0].set_title("up to 9 matches")
axes[0].set_yticklabels(
    [
        "0",
        "500,000",
        "1,000,000",
        "1,500,000",
        "2,000,000",
        "2,500,000",
        "3,000,000",
        "3,500,000",
    ]
)

_start = 10
for i, _end in enumerate([31, 61], start=1):
    counts_by_level.iloc[_start:_end].plot(kind="bar", ax=axes[i])
    axes[i].set_title(f"{_start} to {_end-1} matches")
    _start = _end

i += 1
counts_by_level.iloc[-30:].plot(kind="bar", ax=axes[i])
axes[i].set_title(f"{30} most frequent matches")


axes = axes.reshape((2, 2))

pad = 5  #  in point
for i in range(2):
    axes[-1, i].set_xlabel("Count of number of matches for a peptide")
    axes[i, 0].set_ylabel("number of peptides")

_ = fig.suptitle(
    "Frequency of peptides matched to x items of protein IDs, proteins (combining isotopes) and genes",
    fontsize=16,
)


fig.tight_layout()
_savefig(fig, folder="figures", name="fasta_mapping_counts")

check for homology of sequences in python?

## Create Automated report

- paragraph in tex for report
- summary table

In [ ]:
print(_summary_text)

In [ ]:
os.makedirs(os.path.dirname(SUMMARY_FILE), exist_ok=True)
with open(SUMMARY_FILE, "w") as f:
    f.write(_summary_text)

## Save mappings as JSON

Each `protein_id` is an entry with the following information:
```
'meta': <fasta-header>
'seq': <protein-sequence>
'peptides': <list of list of peptide sequences: [[0-missed-cleavages, 1-missed-cleavage, 2-missed-cleavage]]>
```

In [ ]:
%%time
import json

from config import FN_FASTA_DB

with open(FN_FASTA_DB, "w") as f:
    json.dump(data_fasta, f, indent=4, sort_keys=False)

In [ ]:
os.stat(FN_FASTA_DB).st_size / 1024 / 1024